**Exercice 1 : Frequent pattern mining**

Let « buy » be a table representing transactions (see below). There are 3 attributes: « Tid », « Type » and « Price » exists a functional dependency between Type and Price (a same type of item always corresponds to the same price).

| Tid | Type   | Price |
|-----|--------|-------|
| 1   | Apple  | 2     |
| 1   | Bread  | 4     |
| 1   | Donut  | 3     |
| 1   | Egg    | 4     |
| 2   | Apple  | 2     |
| 2   | Bread  | 4     |
| 2   | Cheese | 1     |
| 2   | Donut  | 3     |
| 2   | Flour  | 4     |
| 3   | Bread  | 4     |
| 3   | Donut  | 3     |
| 3   | Flour  | 4     |
| 4   | Cheese | 1     |
| 4   | Egg    | 4     |
| 4   | Flour  | 4     |

Starting from this dataset, we are interested in association rules: X -> Apple where X is a set of types of item. Let us note that X.Price denotes the multi-set of prices of each type included in X. Then, we can apply an aggregate function (e.g., sum, max or avg) on values X.Price. For instance, {Apple;Bread}.Price equals to {2;4} and then, sum({Apple;Bread}.Price) = 6.

In [24]:
import pandas as pd
import numpy as np
import itertools

data = pd.DataFrame({
    'Tid': [1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4],
    'Type': ['Apple', 'Bread', 'Donut', 'Egg', 'Apple', 'Bread', 'Cheese', 'Donut', 'Flour', 'Bread', 'Donut', 'Flour', 'Cheese', 'Egg', 'Flour'],
    'Price': [2, 4, 3, 4, 2, 4, 1, 3, 4, 4, 3, 4, 1, 4, 4]
})

def calculate_support(itemset, data):
    num_transactions = data['Tid'].nunique()
    support = (data.groupby('Tid')['Type'].apply(set).apply(itemset.issubset).sum()) / num_transactions
    return support

Assuming that the minimal support threshold is 50%, enumerate all the frequent sets of types. Your answer must be represented with a lattice.

In [25]:
min_support = 0.5
frequent_itemsets = []
unique_types = data['Type'].unique()
for r in range(1, len(unique_types) + 1):
    for combo in itertools.combinations(unique_types, r):
        itemset = set(combo)
        support = calculate_support(itemset, data)
        if support >= min_support:
            frequent_itemsets.append((itemset, support))
print("Frequent Sets of Types (Support >= 50%):")
for itemset, support in frequent_itemsets:
    print(f"{itemset}: Support {support:.2f}")

Frequent Sets of Types (Support >= 50%):
{'Apple'}: Support 0.50
{'Bread'}: Support 0.75
{'Donut'}: Support 0.75
{'Egg'}: Support 0.50
{'Cheese'}: Support 0.50
{'Flour'}: Support 0.75
{'Bread', 'Apple'}: Support 0.50
{'Donut', 'Apple'}: Support 0.50
{'Bread', 'Donut'}: Support 0.75
{'Bread', 'Flour'}: Support 0.50
{'Donut', 'Flour'}: Support 0.50
{'Flour', 'Cheese'}: Support 0.50
{'Donut', 'Bread', 'Apple'}: Support 0.50
{'Bread', 'Donut', 'Flour'}: Support 0.50


Find all the confidence of rules X -> Apple which support exceeds 50%. What are the redundant associatio rules?

In [26]:
min_confidence = 0.5
association_rules = []
for itemset, support in frequent_itemsets:
    if 'Apple' in itemset:
        antecedent = itemset.difference(['Apple'])
        confidence = support / calculate_support(antecedent, data)
        if confidence >= min_confidence:
            association_rules.append((antecedent, 'Apple', confidence))
print("\nAssociation Rules (Confidence >= 50%):")
for antecedent, consequent, confidence in association_rules:
    print(f"{antecedent} -> {consequent}: Confidence {confidence:.2f}")


Association Rules (Confidence >= 50%):
set() -> Apple: Confidence 0.50
{'Bread'} -> Apple: Confidence 0.67
{'Donut'} -> Apple: Confidence 0.67
{'Donut', 'Bread'} -> Apple: Confidence 0.67


Now we consider that the constraint is sum(X.Price)≤6 (we do not take into account the support). Which kind of property satisfies this constraint? Find all the sets of types satisfying sum(X.Price)≤6 by building a lattice.

In [27]:
# Calculate the maximum constraint value
max_constraint = 6
# Get unique types and their corresponding prices
unique_types = data[['Type', 'Price']].drop_duplicates()
# Create a dictionary to store type-price pairs
type_price_dict = {}
for _, row in unique_types.iterrows():
    type_price_dict[row['Type']] = row['Price']

def calculate_total_price(itemset, type_price_dict):
    total_price = sum(type_price_dict[type] for type in itemset)
    return total_price

# Generate all possible combinations of types
all_combinations = []
for r in range(1, len(unique_types) + 1):
    combinations = itertools.combinations(unique_types['Type'], r)
    all_combinations.extend(combinations)
# Find sets of types satisfying the constraint sum(X.Price) <= max_constraint
valid_sets = []
for itemset in all_combinations:
    total_price = calculate_total_price(itemset, type_price_dict)
    if total_price <= max_constraint:
        valid_sets.append(itemset)
print("Sets of Types satisfying sum(X.Price) <= 6:")
for itemset in valid_sets:
    print(itemset)

Sets of Types satisfying sum(X.Price) <= 6:
('Apple',)
('Bread',)
('Donut',)
('Egg',)
('Cheese',)
('Flour',)
('Apple', 'Bread')
('Apple', 'Donut')
('Apple', 'Egg')
('Apple', 'Cheese')
('Apple', 'Flour')
('Bread', 'Cheese')
('Donut', 'Cheese')
('Egg', 'Cheese')
('Cheese', 'Flour')
('Apple', 'Donut', 'Cheese')


Now we consider that the constraint is avg(X.Price)≤2.5. What is the difficulty of such kind of constraints? Propose a way enabling to mine the desired sets?

In [28]:
avg_price_constraint = 2.5
filtered_types = []
for itemset, support in frequent_itemsets:
    itemset_prices = data[data['Type'].isin(itemset)]['Price'].values
    avg_price = np.mean(itemset_prices)
    if avg_price <= avg_price_constraint:
        filtered_types.append(itemset)
print("\nSets of Types satisfying avg(X.Price) <= 2.5:")
for itemset in filtered_types:
    print(itemset)


Sets of Types satisfying avg(X.Price) <= 2.5:
{'Apple'}
{'Cheese'}


**Exercice 2 : Lift**

The interestingness measure lift is often used in pattern mining to mine relevant association rules. Given an association rule X->Y where the support of X and Y is not 0, the lift is defined by:
  Lift(X->Y) = Supp(X->Y)/(Supp(X)*Supp(Y))
  
where Supp(X->Y) denotes the support of the rule X->Y and Supp(X) denotes the proportion of transactions containing X.

Rewrite the lift using the confidence of the rule X->Y

In [29]:
def calculate_lift(confidence, support_X, support_Y):
    if support_X == 0 or support_Y == 0:
        return 0  # Lift is undefined in this case
    return confidence / (support_X * support_Y)

# Example values for confidence, support of X, and support of Y
confidence_XY = 0.7  # Confidence of rule X->Y
support_X = 0.5  # Support of itemset X
support_Y = 0.4  # Support of itemset Y
lift_XY = calculate_lift(confidence_XY, support_X, support_Y)
print("Lift(X->Y):", lift_XY)

Lift(X->Y): 3.4999999999999996


Give the range of lift

In [30]:
def interpret_lift(lift):
    if lift == 0:
        return "There is no association between X and Y."
    elif 0 < lift < 1:
        return "X and Y are negatively associated."
    elif lift == 1:
        return "X and Y are independent."
    elif lift > 1:
        return "X and Y are positively associated."

| Tid | Items   |
|-----|--------|
| 1   | ABCD  |
| 2   | ADE  |
| 3   | BDE  |
| 4   | E    |

- Compute the lift of the rules A->set(), A->B and A->BC

In [31]:
transactions = [
    {"Tid": 1, "Items": "ABCD"},
    {"Tid": 2, "Items": "ADE"},
    {"Tid": 3, "Items": "BDE"},
    {"Tid": 4, "Items": "E"},
]
transaction = [[*transaction["Items"]] for transaction in transactions]

def calculate_support(itemset, transaction):
    count = 0
    for t in transaction:
        if all(item in t for item in itemset):
            count += 1
    return count / len(transaction)


# Define the support of individual items
support_A = calculate_support(["A"], transaction)
support_B = calculate_support(["B"], transaction)
support_C = calculate_support(["C"], transaction)
support_D = calculate_support(["D"], transaction)
support_E = calculate_support(["E"], transaction)
# Calculate the confidence of the rules
confidence_A_empty = 1.0  # A->set()
confidence_A_B = calculate_support(["A", "B"], transaction) / support_A  # A->B
confidence_A_BC = calculate_support(["A", "B", "C"], transaction) / support_A  # A->BC
# Calculate the lift of the rules
lift_A_empty = calculate_lift(confidence_A_empty, support_A, 1)  # A->set()
lift_A_B = calculate_lift(confidence_A_B, support_A, support_B)  # A->B
lift_A_BC = calculate_lift(confidence_A_BC, support_A, calculate_support(["B", "C"], transaction))  # A->BC
# Print the lift values
print("Lift(A->set()):", lift_A_empty, interpret_lift(lift_A_empty))
print("Lift(A->B):", lift_A_B, interpret_lift(lift_A_B))
print("Lift(A->BC):", lift_A_BC, interpret_lift(lift_A_BC))

Lift(A->set()): 2.0 X and Y are positively associated.
Lift(A->B): 2.0 X and Y are positively associated.
Lift(A->BC): 4.0 X and Y are positively associated.


Compute all the itemsets of D whose support exceeds 49% and represent them by means of a lattice.

In [32]:
# Calculate the total number of transactions
total_transactions = len(transactions)

def calculate_support(itemset):
    count = sum(1 for transaction in transactions if all(item in transaction["Items"] for item in itemset))
    return count / total_transactions

# Define the support threshold (49%)
support_threshold = 0.49
# Generate all single items
single_items = set(item for transaction in transactions for item in transaction["Items"])
# Initialize frequent itemsets
frequent_itemsets = []
# Generate frequent itemsets
for item in single_items:
    support = calculate_support({item})
    if support >= support_threshold:
        frequent_itemsets.append(({item}, support))
# Generate larger frequent itemsets
for size in range(2, len(single_items) + 1):
    item_combinations = itertools.combinations(single_items, size)
    for itemset in item_combinations:
        support = calculate_support(set(itemset))
        if support >= support_threshold:
            frequent_itemsets.append((set(itemset), support))
# Print frequent itemsets and their support
for itemset, support in frequent_itemsets:
    print(f"Itemset: {itemset}, Support: {support:.2f}")

Itemset: {'B'}, Support: 0.50
Itemset: {'A'}, Support: 0.50
Itemset: {'E'}, Support: 0.75
Itemset: {'D'}, Support: 0.75
Itemset: {'B', 'D'}, Support: 0.50
Itemset: {'D', 'A'}, Support: 0.50
Itemset: {'E', 'D'}, Support: 0.50


In [34]:
import plotly.graph_objs as go

# Create a lattice representation
lattice_data = [f"{', '.join(itemset)} (Support: {support:.2f})" for itemset, support in frequent_itemsets]
# Define the layout and appearance of the lattice
layout = go.Layout(
    title="Lattice Representation of Frequent Itemsets",
    xaxis=dict(title="Frequent Itemsets"),
    yaxis=dict(title="Support Value"),
    showlegend=False,
)
# Create a bar chart for the lattice
fig = go.Figure(data=[go.Bar(x=lattice_data, y=[support for _, support in frequent_itemsets])], layout=layout)
# Customize the appearance and layout of the lattice
fig.update_layout(
    xaxis_title="Frequent Itemsets",
    yaxis_title="Support Value",
    xaxis_tickangle=-45,
)
# Show the lattice plot
fig.show()